In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, lit

In [3]:
spark = SparkSession.builder.master("local[4]").appName("NycTaxi").getOrCreate()

24/11/20 20:19:50 WARN Utils: Your hostname, MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.31.140 instead (on interface en0)
24/11/20 20:19:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/20 20:19:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read.csv("data/yellow_tripdata_2019-01.csv", header=True, inferSchema=True) 
df.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2019-01-01 00:46:40|  2019-01-01 00:53:20|              1|          1.5|         1|                 N|         151|         239|           1|        7.0|  0.5|    0.5|      1.65|         0.0|                  0.3

In [8]:
from pyspark.sql.functions import year, month, dayofmonth
from pyspark.sql.types import StringType

# Extract year, month, and day from tpep_pickup_datetime column
df = df.withColumn("year", year(df.tpep_pickup_datetime).cast(StringType()))
df = df.withColumn("month", month(df.tpep_pickup_datetime).cast(StringType()))
df = df.withColumn("day", dayofmonth(df.tpep_pickup_datetime).cast(StringType()))

# Save the DataFrame as Parquet files partitioned by year, month, and day


+----+
|year|
+----+
|2019|
|2018|
|2009|
|2008|
|2003|
|2001|
|2088|
+----+



In [15]:
nd = df.filter(~(df.year == '2019'))
nd.count()

443

24/11/21 02:08:52 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 936163 ms exceeds timeout 120000 ms
24/11/21 02:08:52 WARN SparkContext: Killing executors is not supported by current scheduler.
24/11/21 02:26:39 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import lag, col

# Create Spark session
spark = SparkSession.builder.appName("VaccinationAnalysis").getOrCreate()

# Sample data
data = [
    ("A", 2020, "No", "No"),
    ("A", 2021, "Yes", "Yes"),
    ("A", 2022, "Yes", "No"),
    ("B", 2020, "No", "Yes"),
    ("B", 2021, "Yes", "No"),
    ("B", 2022, "Yes", "Yes"),
    ("C", 2020, "Yes", "Yes"),
    ("C", 2021, "No", "No"),
    ("C", 2022, "Yes", "Yes"),
    ("D", 2020, "Yes", "No"),
    ("D", 2021, "Yes", "Yes"),
    ("D", 2022, "Yes", "No"),
]

columns = ["empname", "year", "vaccination", "infected"]

# Create DataFrame
df = spark.createDataFrame(data, schema=columns)

# Define a window specification
window_spec = Window.partitionBy("empname").orderBy("year")

# Use lag to get vaccination status and infection status from the previous year
df_with_lag = df.withColumn("previous_vaccination", lag("vaccination").over(window_spec)) \
                .withColumn("previous_year_infected", lag("infected").over(window_spec))

# Filter employees who were vaccinated in a year and infected the next year
result = df_with_lag.filter((col("previous_vaccination") == "Yes") & (col("infected") == "Yes"))

# Select distinct employee names
result = result.select("empname").distinct()

# Show the result
result.show()

24/12/03 23:06:15 WARN Utils: Your hostname, MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.104 instead (on interface en0)
24/12/03 23:06:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/03 23:06:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-------+
|empname|
+-------+
|      B|
|      D|
+-------+



In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import least, greatest

# Create a Spark session
spark = SparkSession.builder.appName("UniqueCityPairs").getOrCreate()

# Sample data
data = [
    ("A", "B", 10),
    ("B", "A", 10),
    ("A", "C", 15),
    ("C", "A", 15),
    ("B", "C", 20),
    ("C", "B", 20),
]

# Define schema
columns = ["city1", "city2", "distance"]

# Create DataFrame
df = spark.createDataFrame(data, schema=columns)

# Use least and greatest to standardize the city pairs
df_unique_pairs = df.withColumn("city1_standard", least("city1", "city2")) \
                    .withColumn("city2_standard", greatest("city1", "city2")) \
                    .select("city1_standard", "city2_standard", "distance") \
                    .distinct()

# Rename columns for clarity
df_result = df_unique_pairs.withColumnRenamed("city1_standard", "city1") \
                            .withColumnRenamed("city2_standard", "city2")

# Show the result
df_result.show()

24/12/03 23:11:32 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+-----+-----+--------+
|city1|city2|distance|
+-----+-----+--------+
|    A|    B|      10|
|    A|    C|      15|
|    B|    C|      20|
+-----+-----+--------+

